# Python AST Module 
- Environment: python3.9 
- Author: Ching Wen Yang
- Created: 2023-06-16 20:40 


In [1]:
import ast 
expression = "n=10"
node = ast.parse(expression, mode='exec')
# 取得 string 描述
ast.dump(node)

"Module(body=[Assign(targets=[Name(id='n', ctx=Store())], value=Constant(value=10))], type_ignores=[])"

> As part of the internship, we'll be using Python AST to interpret custom logical expressions. To prepare, you may want to use Python AST to parser basic logical expressions and then explore how to customize your tokens.
- References
[gyermolenko/awesome-python-ast](https://github.com/gyermolenko/awesome-python-ast)

## AST Document Structure 
- ast — Abstract Syntax Trees
- Abstract Grammar
- Node classes
  - Literals
  - Variables
    - Expressions
    - Subscripting
    - Comprehensions
  - Statements
    - Imports
  - Control flow
  - Pattern matching
  - Function and class definitions
  - Async and await
- ast Helpers
- Compiler Flags
- Command-Line Usage

## [Node Classes](https://docs.python.org/zh-tw/3/library/ast.html#node-classes)
### 字面值 Literals 
- `ast.Constant`: 一個常量，數字、字串、None對象、frozenset or Tuple。
- `ast.FormattedValue`: 一個格式化的值，由 f-string 字串格式化而來。比如f'{x}'。
- `ast.JoinedStr`: 一個f-string，可以是 Constant 或 FormattedValue 多個組合而成。
- `ast.List`, `ast.Tuple`: 一個列表或元組。 elts 保存內容，ctx 為 Load, Store。ctx 在列表或元組中分辨在左值或右值。 
  - `(x, y) = fn(results)` 中，`(x,y)` 的 ctx 為 Store。
- `ast.Set`: 一個集合。 elts 保存內容。
- `ast.Dict`: 一個字典。 keys 保存鍵，values 保存值。 
### 變數 Variables 
- `ast.Name(id, ctx)`：一個變數，id 為變數名，ctx 為 `ast.Load, ast.Store, ast.Del`。ctx 在變數中分辨在左值或右值。 
- The above 3 ctx classes
- `ast.Starred(value, ctx)`: A *var variable reference.

### 表現式 Expressions
- `ast.Expr(value)`: 如 function call 這樣的表現式被呼叫但是 return 未被接住的情況，這個 return value 會被存在這個 Expr 內。
- `ast.UnaryOp(op, operand)`: 一個一元運算符，op 為運算符，operand 為運算元。eg. -1, not True
- Unary operator tokens 
    - `ast.UAdd`: + 
    - `ast.USub`: - 
    - `ast.Not`: not keyword 
    - `ast.Invert`: ~
- `ast.BinOp(left, op, right)`: 一個二元運算符，left 為左運算元，op 為運算符，right 為右運算元。eg. 1 + 2
- Binary operator tokens 
    - `ast.Add`: + 
    - `ast.Sub`: - 
    - `ast.Mult`: * 
    - `ast.Div`: / 
    - `ast.FloorDiv`: // 
    - `ast.Mod`: % 
    - `ast.Pow`: ** 
    - `ast.LShift`: << 
    - `ast.RShift`: >> 
    - `ast.BitOr`: | 
    - `ast.BitXor`: ^ 
    - `ast.BitAnd`: & 
    - `ast.MatMult`: @ 
- `ast.BoolOp(op, values)`: 一個布林運算符，op 為運算符，values 為運算元。eg. True and False
- Boolean operator tokens 
    - `ast.And`: and keyword 
    - `ast.Or`: or keyword

```python
code = """
def f(x):
    return x + 1
f(x) 

"""
print(ast.dump(ast.parse(code, mode='exec'), indent=4))
>>> Expr(...
    value=Call(
        func=Name(id='f', ctx=Load()),
        args=[
            Name(id='x', ctx=Load())],
        keywords=[]))],...) 
```


In [24]:
# expression 
code = """
def f(x):
    return x + 1
f(x) 

"""
print(ast.dump(ast.parse(code, mode='exec'), indent=4))

Module(
    body=[
        FunctionDef(
            name='f',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Return(
                    value=BinOp(
                        left=Name(id='x', ctx=Load()),
                        op=Add(),
                        right=Constant(value=1)))],
            decorator_list=[]),
        Expr(
            value=Call(
                func=Name(id='f', ctx=Load()),
                args=[
                    Name(id='x', ctx=Load())],
                keywords=[]))],
    type_ignores=[])


In [20]:
from math import sin
a = 8 
joinedstr_ex = f"sin({a}) is {sin(a):.3}"
print(ast.dump(ast.parse(joinedstr_ex, mode='eval'), indent=4))
# del 
print(ast.dump(ast.parse("del a", mode='exec'), indent=4))

Expression(
    body=Compare(
        left=Call(
            func=Name(id='sin', ctx=Load()),
            args=[
                Constant(value=8)],
            keywords=[]),
        ops=[
            Is()],
        comparators=[
            Constant(value=0.989)]))
Module(
    body=[
        Delete(
            targets=[
                Name(id='a', ctx=Del())])],
    type_ignores=[])


In [22]:
a, *b = iter([1,2,3])
print(a) 
print(b)
print(ast.dump(ast.parse("a, *b = iter([1,2,3])", mode='exec'), indent=4))

1
[2, 3]
Module(
    body=[
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='a', ctx=Store()),
                        Starred(
                            value=Name(id='b', ctx=Store()),
                            ctx=Store())],
                    ctx=Store())],
            value=Call(
                func=Name(id='iter', ctx=Load()),
                args=[
                    List(
                        elts=[
                            Constant(value=1),
                            Constant(value=2),
                            Constant(value=3)],
                        ctx=Load())],
                keywords=[]))],
    type_ignores=[])


## Parse a code string into an AST 

In [11]:
code_string = """

def add(a, b):
    return a + b

def sub(a, b):
    return a - b

res = add(1, 2)
add(1, 2) == sub(3, 2)
print(res)
"""

tree = ast.parse(code_string)
print(ast.dump(tree, indent=4))

Module(
    body=[
        FunctionDef(
            name='add',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a'),
                    arg(arg='b')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Return(
                    value=BinOp(
                        left=Name(id='a', ctx=Load()),
                        op=Add(),
                        right=Name(id='b', ctx=Load())))],
            decorator_list=[]),
        FunctionDef(
            name='sub',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a'),
                    arg(arg='b')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Return(
                    value=BinOp(
                        left=Name(id='a', ctx=Load()),
              

### NodeVisitor 
`class ast.NodeVisitor`
一个遍历抽象语法树并针对所找到的每个节点调用访问器函数的节点访问器基类。 该函数可能会返回一个由 visit() 方法所提供的值。
这个类应当被子类化，并由子类来添加访问器方法。
- `visit(node)`
访问一个节点。 默认实现会调用名为 `self.visit_classname` 的方法其中 classname 为节点类的名称，或者如果该方法不存在则为 `generic_visit()`。

- `generic_visit(node)`
该访问器会在节点的所有子节点上调用 `visit()`。

请注意所有包含自定义访问器方法的节点的子节点将不会被访问除非访问器调用了 `generic_visit()` 或是自行访问它们。

如果你想在遍历期间应用对节点的修改则请不要使用 `NodeVisitor`。 对此目的可使用一个允许修改的特殊访问器 (`NodeTransformer`)。

在 3.8 版本開始棄用: visit_Num(), visit_Str(), visit_Bytes(), visit_NameConstant() 和 visit_Ellipsis() 等方法现在已被弃用且在未来的 Python 版本中将不会再被调用。 
请添加 visit_Constant() 方法来处理所有常量节点。

In [17]:
# How to navigate and manipulate nodes in the AST
# takeaway: subclass ast.NodeVisitor and implement visit_<nodename> methods 
# 1. traverse 
from _ast import Eq
from typing import Any


class FnNameVisitor(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print(node.name)
        self.generic_visit(node) # designed to visit the children nodes of the current node 
        # ensure that all nodes from the given `node` are visited

class ConstantVisitor(ast.NodeVisitor):
    def visit_Constant(self, node):
        print(node._fields)
        print(node.value)
        print(node.kind)
        self.generic_visit(node)

# prints the names of all functions in the code 
tree = ast.parse(code_string)
FnNameVisitor().visit(tree)
ConstantVisitor().visit(tree)


# prints the equality operator

add
sub
('value', 'kind')
1
None
('value', 'kind')
2
None
('value', 'kind')
1
None
('value', 'kind')
2
None
('value', 'kind')
3
None
('value', 'kind')
2
None


## Visualize an AST 
Reference: https://jckling.github.io/2021/07/14/Other/Python%20ast%20%E6%A8%A1%E5%9D%97%E4%BD%BF%E7%94%A8/